In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2

import numpy as np
from collections import defaultdict


from PIL import Image, ImageFilter
import io
import re
import random
import numpy.random as npr
from skimage import data
from scipy.ndimage import rotate
from kernels import *
import torchvision
from torchvision import models
import os
from torchvision.transforms.functional import to_pil_image
from torch.utils.data import Dataset, DataLoader, Subset

import torchvision.transforms as transforms
 
import my_utils as ut
import old_utils_averaged_filters as old_ut_avg
import old_utils_multi_image_old_snr as old_ut_multi
from transformers import Swinv2ForImageClassification, SwinConfig
from torch.optim import AdamW
from torchvision import transforms, datasets



In [2]:

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
   
])

# Create dataset instances
train_dataset = old_ut_multi.DatasetAI(root_dir='/mnt/e/GenImage', transform=transform, split='train')
val_test_dataset = old_ut_multi.DatasetAI(root_dir='/mnt/e/GenImage', transform=transform, split='val')


train_subset,val_subset, test_subset = ut.split_datasets(train_dataset, val_test_dataset, 30000, 3750, 3750)





train_loader = DataLoader(train_subset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_subset, batch_size=32, shuffle=False, num_workers=4)
test_loader = DataLoader(test_subset, batch_size=32, shuffle=False, num_workers=4)



In [3]:
#check distribution of classes 

print("Check overlap between datasets:\n")
print("\nTrain Subset and Validation Subset:")
ut.check_data_overlap(train_subset, val_subset)
print("\nTrain Subset and Test Subset:")
ut.check_data_overlap(train_subset, test_subset)
print("\nValidation Subset and Test Subset:")
ut.check_data_overlap(val_subset, test_subset)

print()
print("\nTrain Subset Distribution:\n")
ut.print_model_class_distribution(train_dataset, train_subset.indices)
print("\nValidation Subset Distribution:\n")
ut.print_model_class_distribution(val_test_dataset, val_subset.indices)
print("\nTest Subset Distribution:\n")
ut.print_model_class_distribution(val_test_dataset, test_subset.indices)

Check overlap between datasets:


Train Subset and Validation Subset:
No actual data overlap detected.

Train Subset and Test Subset:
No actual data overlap detected.

Validation Subset and Test Subset:
No actual data overlap detected.


Train Subset Distribution:

Total samples in subset: 30000
Model ADM, Class nature: 1815 (6.05%)
Model ADM, Class ai: 1935 (6.45%)
Model BigGAN, Class ai: 1867 (6.22%)
Model BigGAN, Class nature: 1883 (6.28%)
Model Midjourney, Class ai: 1884 (6.28%)
Model Midjourney, Class nature: 1866 (6.22%)
Model VQDM, Class ai: 1810 (6.03%)
Model VQDM, Class nature: 1940 (6.47%)
Model glide, Class ai: 1891 (6.30%)
Model glide, Class nature: 1859 (6.20%)
Model stable_diffusion_v_1_4, Class ai: 1892 (6.31%)
Model stable_diffusion_v_1_4, Class nature: 1858 (6.19%)
Model stable_diffusion_v_1_5, Class nature: 1828 (6.09%)
Model stable_diffusion_v_1_5, Class ai: 1922 (6.41%)
Model wukong, Class nature: 1852 (6.17%)
Model wukong, Class ai: 1898 (6.33%)

Validation Subset 

## old smash and reconstruct 

In [4]:
class BaseClassifier(nn.Module):
    def __init__(self,kernels):
        super(BaseClassifier, self).__init__()
        self.feature_combiner = ut.CNNBlock(kernels)
        self.feature_combiner2 = ut.CNNBlock(kernels)
        self.classifier = ut.DeepClassifier() 

 
    def forward(self, rich, poor):
       
        x = self.feature_combiner(rich)
        y = self.feature_combiner2(poor)   
        feature_difference = x - y
        outputs = self.classifier(feature_difference)

        return outputs
class ResNetClassifier(nn.Module):
    def __init__(self,kernels):
        super(ResNetClassifier, self).__init__()
        self.feature_combiner = ut.CNNBlock(kernels)
        self.feature_combiner2 = ut.CNNBlock(kernels)
        resnet_weights = models.ResNet50_Weights.DEFAULT
        self.resnet = models.resnet50(weights=resnet_weights)
        # Remove the final fully connected layer
        self.features = nn.Sequential(*list(self.resnet.children())[:-2])
        # Add a new classifier layer
        self.classifier = nn.Linear(self.resnet.fc.in_features, 2)
        # Adaptive pool to make sure output from feature maps is of fixed size
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))

 
    def forward(self, rich, poor):
       
        x = self.feature_combiner(rich)
        y = self.feature_combiner2(poor)   
        feature_difference = x - y
        
        # Process feature difference through the ResNet features
        features = self.features(feature_difference)
        pooled_features = self.adaptive_pool(features)
        flat_features = torch.flatten(pooled_features, 1)
        outputs = self.classifier(flat_features)

        return outputs
class SwinClassification(nn.Module):
    def __init__(self,kernels):
        super(SwinClassification, self).__init__()
        self.feature_combiner = ut.CNNBlock(kernels)
        self.feature_combiner2 = ut.CNNBlock(kernels)
        config = SwinConfig.from_pretrained('microsoft/swinv2-tiny-patch4-window8-256',num_classes=2)
        self.transformer = Swinv2ForImageClassification.from_pretrained(
            "microsoft/swinv2-tiny-patch4-window8-256",
            config=config
        )
        
        self.transformer.classifier = nn.Linear(config.hidden_size, 2) 

 
    def forward(self, rich, poor):
       
        x = self.feature_combiner(rich)
        y = self.feature_combiner2(poor)   
        feature_difference = x - y
        outputs = self.transformer(feature_difference)

        return outputs.logits


# Frozen models old smash and reconstruct

In [5]:
#Swin Transformer
learning_rates = [1e-3,1e-4, 1e-5, 1e-6]
print("Training SWIN Transformer Frozen w/OLDSNR\n")

for learning_rate in learning_rates:
    print(f"Training with Learning Rate: {learning_rate} \n\n")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    kernels =  ut.apply_high_pass_filter()
    model = SwinClassification(kernels).to(device)
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    best_model_path = '/home/kosta/code/School/SentryAI/pth/best_model_oldSMR_SWIN_frozen.pth'

    #freeze the transformer layers
    for param in model.transformer.parameters():
        param.requires_grad = False
    #unfreeze the classifier layer
    for param in model.transformer.classifier.parameters():
        param.requires_grad = True

    ut.train_and_validate(model, train_loader, val_loader, optimizer, device, num_epochs=10, best_model_path= best_model_path)



Training SWIN Transformer Frozen w/OLDSNR

Training with Learning Rate: 0.001 




You are using a model of type swinv2 to instantiate a model of type swin. This is not supported for all configurations of models and can yield errors.


No saved model found. Starting fresh!


Epoch 1/10
,Train Loss: 0.5943, Train Acc: 0.6789, Val Loss: 0.5347, Val Acc: 0.7209

Saved new best model with accuracy: 0.7209


Epoch 2/10
,Train Loss: 0.5528, Train Acc: 0.7147, Val Loss: 0.4898, Val Acc: 0.7585

Saved new best model with accuracy: 0.7585


...

Epoch 6/10
,Train Loss: 0.6780, Train Acc: 0.5787, Val Loss: 0.6567, Val Acc: 0.6322



Epoch 7/10
,Train Loss: 0.6727, Train Acc: 0.5909, Val Loss: 0.6517, Val Acc: 0.6477



Epoch 8/10
,Train Loss: 0.6690, Train Acc: 0.6003, Val Loss: 0.6454, Val Acc: 0.6501



Epoch 9/10
,Train Loss: 0.6649, Train Acc: 0.6032, Val Loss: 0.6413, Val Acc: 0.6504



Epoch 10/10
,Train Loss: 0.6601, Train Acc: 0.6137, Val Loss: 0.6354, Val Acc: 0.6624



In [6]:
#ResNET
learning_rates = [1e-3,1e-4, 1e-5, 1e-6]
print("Training RES Frozen w/OLDSNR\n")

for learning_rate in learning_rates:
    print(f"Training with Learning Rate: {learning_rate} \n\n")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = ResNetClassifier(kernels).to(device)
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    best_model_path = '/home/kosta/code/School/SentryAI/pth/best_model_oldSMR_RESNET_frozen.pth'
    
    #freeze resnet layers
    for param in model.resnet.parameters():
        param.requires_grad = False
    #unfreeze the classifier layer
    for param in model.classifier.parameters():
        param.requires_grad = True
    ut.train_and_validate(model, train_loader, val_loader, optimizer, device, num_epochs=10, best_model_path= best_model_path)
    
    

Training RES Frozen w/OLDSNR

Training with Learning Rate: 0.001 




No saved model found. Starting fresh!


Epoch 1/10
,Train Loss: 0.5553, Train Acc: 0.7152, Val Loss: 0.4974, Val Acc: 0.7634

Saved new best model with accuracy: 0.7634


Epoch 2/10
,Train Loss: 0.5246, Train Acc: 0.7424, Val Loss: 0.4942, Val Acc: 0.7716

Saved new best model with accuracy: 0.7716


Epoch 3/10
,Train Loss: 0.5118, Train Acc: 0.7507, Val Loss: 0.4952, Val Acc: 0.7524



...

Epoch 6/10
,Train Loss: 0.6562, Train Acc: 0.6422, Val Loss: 0.6444, Val Acc: 0.6715



Epoch 7/10
,Train Loss: 0.6519, Train Acc: 0.6437, Val Loss: 0.6416, Val Acc: 0.6707



Epoch 8/10
,Train Loss: 0.6480, Train Acc: 0.6522, Val Loss: 0.6354, Val Acc: 0.6870



Epoch 9/10
,Train Loss: 0.6443, Train Acc: 0.6534, Val Loss: 0.6291, Val Acc: 0.6854



Epoch 10/10
,Train Loss: 0.6405, Train Acc: 0.6579, Val Loss: 0.6294, Val Acc: 0.6838



# unfreeze the pretrained models old reconstruct and destroy

In [7]:
#ResNET
print("Training RES UNFrozen w/OLDSNR\n")

learning_rates = [1e-3,1e-4, 1e-5, 1e-6]

for learning_rate in learning_rates:
    print(f"Training with Learning Rate: {learning_rate} \n\n")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = ResNetClassifier(kernels).to(device)
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    best_model_path = '/home/kosta/code/School/SentryAI/pth/best_model_oldSMR_RESNET_unfrozen.pth'


    ut.train_and_validate(model, train_loader, val_loader, optimizer, device, num_epochs=10, best_model_path= best_model_path)



Training RES UNFrozen w/OLDSNR

Training with Learning Rate: 0.001 




No saved model found. Starting fresh!


Epoch 1/10
,Train Loss: 0.5019, Train Acc: 0.7562, Val Loss: 0.3991, Val Acc: 0.8315

Saved new best model with accuracy: 0.8315


Epoch 2/10
,Train Loss: 0.3308, Train Acc: 0.8594, Val Loss: 0.6497, Val Acc: 0.7006



Epoch 3/10
,Train Loss: 0.2613, Train Acc: 0.8902, Val Loss: 0.2585, Val Acc: 0.8969

Saved new best model with accuracy: 0.8969


...

Epoch 6/10
,Train Loss: 0.5339, Train Acc: 0.7334, Val Loss: 0.5063, Val Acc: 0.7535



Epoch 7/10
,Train Loss: 0.5233, Train Acc: 0.7372, Val Loss: 0.4943, Val Acc: 0.7529



Epoch 8/10
,Train Loss: 0.5102, Train Acc: 0.7457, Val Loss: 0.4910, Val Acc: 0.7620



Epoch 9/10
,Train Loss: 0.5020, Train Acc: 0.7548, Val Loss: 0.4865, Val Acc: 0.7585



Epoch 10/10
,Train Loss: 0.4914, Train Acc: 0.7608, Val Loss: 0.4727, Val Acc: 0.7722



In [8]:
#Swin Transformer
print("Training SWIN UNFrozen w/OLDSNR\n")

learning_rates = [1e-3,1e-4, 1e-5, 1e-6]

for learning_rate in learning_rates:
    print(f"Training with Learning Rate: {learning_rate} \n\n")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    kernels =  ut.apply_high_pass_filter()
    model = SwinClassification(kernels).to(device)
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    best_model_path = '/home/kosta/code/School/SentryAI/pth/best_model_oldSMR_SWIN_Unfrozen.pth'
    
    
    
    ut.train_and_validate(model, train_loader, val_loader, optimizer, device, num_epochs=10, best_model_path= best_model_path)



Training SWIN UNFrozen w/OLDSNR

Training with Learning Rate: 0.001 




You are using a model of type swinv2 to instantiate a model of type swin. This is not supported for all configurations of models and can yield errors.


No saved model found. Starting fresh!


Epoch 1/10
,Train Loss: 0.6845, Train Acc: 0.5533, Val Loss: 0.6459, Val Acc: 0.6151

Saved new best model with accuracy: 0.6151


Epoch 2/10
,Train Loss: 0.6765, Train Acc: 0.5683, Val Loss: 0.6918, Val Acc: 0.5772



...

Epoch 6/10
,Train Loss: 0.4372, Train Acc: 0.7902, Val Loss: 0.3886, Val Acc: 0.8138



Epoch 7/10
,Train Loss: 0.4193, Train Acc: 0.8022, Val Loss: 0.3709, Val Acc: 0.8272



Epoch 8/10
,Train Loss: 0.4050, Train Acc: 0.8099, Val Loss: 0.3662, Val Acc: 0.8253



Epoch 9/10
,Train Loss: 0.3939, Train Acc: 0.8175, Val Loss: 0.3446, Val Acc: 0.8379



Epoch 10/10
,Train Loss: 0.3821, Train Acc: 0.8244, Val Loss: 0.3413, Val Acc: 0.8456



In [9]:
#Original Model
print("Training BASE w/OLDSNR\n")

learning_rates = [1e-3,1e-4, 1e-5, 1e-6]

for learning_rate in learning_rates:
    print(f"Training with Learning Rate: {learning_rate} \n\n")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = BaseClassifier(kernels).to(device)
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    best_model_path = '/home/kosta/code/School/SentryAI/pth/best_model_oldSMR_Base.pth'


    ut.train_and_validate(model, train_loader, val_loader, optimizer, device, num_epochs=10, best_model_path= best_model_path)



Training BASE w/OLDSNR

Training with Learning Rate: 0.001 


No saved model found. Starting fresh!


Epoch 1/10
,Train Loss: 0.6141, Train Acc: 0.6610, Val Loss: 0.6182, Val Acc: 0.6677

Saved new best model with accuracy: 0.6677


Epoch 2/10
,Train Loss: 0.5522, Train Acc: 0.7239, Val Loss: 0.5978, Val Acc: 0.6811

Saved new best model with accuracy: 0.6811


Epoch 3/10
,Train Loss: 0.5063, Train Acc: 0.7587, Val Loss: 0.4835, Val Acc: 0.7767

Saved new best model with accuracy: 0.7767


Epoch 4/10
,Train Loss: 0.4531, Train Acc: 0.7959, Val Loss: 0.4297, Val Acc: 0.7994

Saved new best model with accuracy: 0.7994


...

Epoch 6/10
,Train Loss: 0.6265, Train Acc: 0.6489, Val Loss: 0.6009, Val Acc: 0.6760



Epoch 7/10
,Train Loss: 0.6223, Train Acc: 0.6520, Val Loss: 0.5953, Val Acc: 0.6787



Epoch 8/10
,Train Loss: 0.6177, Train Acc: 0.6550, Val Loss: 0.5898, Val Acc: 0.6867



Epoch 9/10
,Train Loss: 0.6153, Train Acc: 0.6554, Val Loss: 0.5880, Val Acc: 0.6854



Epoch 10/10
,Train Loss: 0.6140, Train Acc: 0.6569, Val Loss: 0.5822, Val Acc: 0.6910

